# Agno

[Agno](agno.com) es un framework que pretende aligerar la forma en la que otros frameworks nos obligan a trabajar para crear nuestros agentes. Simplifica gran parte del código a generar centrándose en los aspectos clave. Ofrece una plataforma donde a futuro será posible gestionar nuestros agentes y flujos de trabajo (https://app.agno.com/) aunque de momento lo podemos emplear como framework local. Dispone de multitud de ejemplos en la [documentación](https://docs.agno.com/).

![](https://mintcdn.com/agno/QZOB15dhrj4yAmBd/images/workspace.png?w=840&maxW=3034&auto=format&n=QZOB15dhrj4yAmBd&q=85&s=192feab94035c340f257f7b7f228cd19)

Podemos levantar todo un workspace local con ejemplos siguiendo los pasos en: https://docs.agno.com/workspaces/introduction

Aunque para estos ejemplos iremos paso a paso, empezando con la importación de las claves de conexión y algunos parámetros básicos.

In [35]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
from agno.agent import Agent
from agno.models.google import Gemini

# Create a News Reporter Agent with a fun personality
agent = Agent(
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    instructions="Eres un asistente de matemáticas.",
    show_tool_calls=True,
    markdown=True,
)

# Example usage
agent.print_response(
    "Hola, ¿puedes ayudarme con unos cálculos?", stream=True
)

Output()

In [3]:
agent.print_response(
    "¿Cuanto es 4 * 5?", stream=True
)

Output()

Al igual que LangChain podemos trazar la actividad pero veréis que en este caso es algo menos visual en [LangSmith via OpenTelemetry](https://docs.agno.com/examples/concepts/observability/langsmith-via-openinference)

In [4]:
import os
from openinference.instrumentation.agno import AgnoInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

# Set the endpoint and headers for LangSmith
endpoint = "https://api.smith.langchain.com/otel/v1/traces"
headers = {
    "x-api-key": os.getenv("LANGSMITH_API_KEY"),
    "Langsmith-Project": os.getenv("LANGSMITH_PROJECT"),
}

# Configure the tracer provider
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(
    SimpleSpanProcessor(OTLPSpanExporter(endpoint=endpoint, headers=headers))
)
trace_api.set_tracer_provider(tracer_provider=tracer_provider)

# Start instrumenting agno
AgnoInstrumentor().instrument()

Podemos extender el uso de herramientas tal y como vimos con LangChain.

In [5]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

In [7]:
# Create a News Reporter Agent with a fun personality
agent = Agent(
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    instructions="Eres un asistente de matemáticas.",
    tools=[multiply],
    show_tool_calls=True,
    markdown=True,
)

agent.print_response(
    "¿Cuanto es 4 * 5?", stream=True
)

Output()

Podemos incluir herramientas más elaboradas como los buscadores web.

In [10]:
from agno.tools.tavily import TavilyTools

# Search engine
search = TavilyTools()

agent = Agent(
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    instructions="""
        Eres un especialista en viajes trabajando en una agencia. 
        Tu trabajo es proponer opciones de viajes a los clientes aunque tienes una predilección por Bilbao y siempre que puedes lo propones como destino.""",
    tools=[search],
    show_tool_calls=True,
    markdown=True,
)

agent.print_response(
    "¿Qué viajes hay a Madrid desde Santander?", stream=True
)

Output()

In [13]:
agent.print_response(
    "Está bien, busca entonces vuelos a ese destino", stream=True
)

Output()

Acordémonos de que los agentes no tienen memoria. Necesitamos indicarle cuantos mensajes debe recordar.

In [17]:
agent = Agent(
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    instructions="""
        Eres un especialista en viajes trabajando en una agencia. 
        Tu trabajo es proponer opciones de viajes a los clientes aunque tienes una predilección por Bilbao y siempre que puedes lo propones como destino.""",
    tools=[search],
    # Memoria
    add_history_to_messages=True,
    num_history_responses=3,
    show_tool_calls=True,
    markdown=True,
)

agent.print_response(
    "Estoy en Santander ¿Qué viajes hay a Madrid?", stream=True
)

Output()

In [18]:
agent.print_response(
    "Está bien, busca entonces vuelos a ese destino desde mi ciudad", stream=True
)

Output()

En muchos casos necesitaremos que nuestro interlocutor nos de el ok a la operación. Para eso, debemos instruir al agente de que la ejecución de la herramienta debe disponer de una aceptación.

In [19]:
import json
from typing import Any, Callable, Dict, Iterator

import httpx
from agno.exceptions import StopAgentRun
from agno.tools import tool
from rich.console import Console
from rich.prompt import Prompt

# Consola
console = Console()

# Confirmation hook
def confirmation_hook(
    function_name: str, function_call: Callable, arguments: Dict[str, Any]
):
    # Get the live display instance from the console
    live = console._live

    # Stop the live display temporarily so we can ask for user confirmation
    live.stop()  # type: ignore

    # Ask for confirmation
    console.print(f"\nVoy a ejecutar [bold blue]{function_name}[/]")
    message = (
        Prompt.ask("¿Quieres que proceda?", choices=["s", "n"], default="s")
        .strip()
        .lower()
    )

    # Restart the live display
    live.start()  # type: ignore

    # If the user does not want to continue, raise a StopExecution exception
    if message != "s":
        raise StopAgentRun(
            "Tool call cancelled by user",
            agent_message="Stopping execution as permission was not granted.",
        )
    
    # Call the function
    result = function_call(**arguments)

    # Optionally transform the result

    return result

# A tool that requests confirmation
@tool(tool_hooks=[confirmation_hook])
def get_top_hackernews_stories(num_stories: int) -> Iterator[str]:
    """Fetch top stories from Hacker News.

    Args:
        num_stories (int): Number of stories to retrieve

    Returns:
        str: JSON string containing story details
    """
    # Fetch top story IDs
    response = httpx.get("https://hacker-news.firebaseio.com/v0/topstories.json")
    story_ids = response.json()

    # Yield story details
    final_stories = []
    for story_id in story_ids[:num_stories]:
        story_response = httpx.get(
            f"https://hacker-news.firebaseio.com/v0/item/{story_id}.json"
        )
        story = story_response.json()
        if "text" in story:
            story.pop("text", None)
        final_stories.append(story)

    return json.dumps(final_stories)

In [20]:
from agno.agent import Agent
from agno.models.google import Gemini

# Create an Agent
agent = Agent(
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    instructions="Eres un especialista en periodismo tecnológico y puedes predecir tendencias de mercado basado en noticias de hackernews.",
    tools=[get_top_hackernews_stories],
    show_tool_calls=True,
    markdown=True,
)

agent.print_response(
    "¿Qué disrupciones prevés para este final de año?", stream=True, console=console
)

Output()

Voy a ejecutar get_top_hackernews_stories

¿Quieres que proceda? [s/n] (s):

## Retrieve-Augmented Generation (RAG)

Más allá de las herramientas, uno de los pasos clave es cuando podemos introducir información curada o específica de nuestro dominio. Debemos encontrar la mejor forma de hacerlo pero podemos disponer de bases de datos que nos permitan facilitar era búsqueda y que la información obtenida sea parte de nuestra consulta. Este mecanismo es el conocido como RAG. El equipo de Pinecone, una de las primeras bases de datos vectoriales en su momento, tiene una [entrada de blog](https://www.pinecone.io/learn/retrieval-augmented-generation/) bastante aclaratoria de los sistemas RAG.

![](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2Ff6fe392bb5287791a2c6052f1eeb3072ad0b7e36-2236x2620.png&w=3840&q=75)

Dependiendo del tipo de pregunta que vayamos a hacer podemos necesitar un sistema distinto al que habitualmente tenemos (RDBMS):

* Para búsquedas difusas o resumen de una temática, precisaremos buscar textos que hablen de esta por **similitud semántica**.
* A veces necesitamos la ocurrencia exacta de términos, y esto podemos hacerlo mediante **filtros explícitos**.

En otros casos necesitaremos conocer la estructura vinculada de los textos (legal) o jerarquía de forma que textos que no hablan del mismo concepto o no presentan términos concretos habiliten distintas acepciones. De ahí que Microsoft propusiera en [2024 el GraphRAG](https://microsoft.github.io/graphrag/) basado en bases de datos en grafos.

La gran tarea de estos sistemas es precisamente modelar la información para poder luego interrogarla. En el caso de bases de datos vectoriales:

* **seccionado de la información** https://www.pinecone.io/learn/chunking-strategies/
* **embedding**
* **establecer el modelo de búsqueda** mediante búsqueda semántica y lexicológica lo que conocemos como búsqueda híbrida (https://docs.pinecone.io/guides/search/hybrid-search)

Quizás por ser los primeros en el mercado tiene una posición predominante aunque otros proveedores (LanceDB, Chroma, Qdrant, etc.) presentan una oferta similar. Podéis ver los sistemas más usados en https://db-engines.com/en/ranking_trend/vector+dbms

Desde el punto de vista académico, Facebook con [FAISS](https://faiss.ai/) fue una de las primeras empresas en plantear este tipo de búsqueda de información.

En el ejemplo de abajo emplearemos [LanceDB](https://www.lancedb.com/) que está disponible tanto en formato on-premise como cloud.


In [24]:
from agno.embedder.google import GeminiEmbedder
from agno.vectordb.lancedb import LanceDb, SearchType
from agno.knowledge.markdown import MarkdownKnowledgeBase

# Database
vector_db = LanceDb(
    table_name="langchain",
    uri="./lancedb",  # You can change this path to store data elsewhere
    embedder=GeminiEmbedder(),
    use_tantivy=True
)

# Knowledge base
knowledge_base = MarkdownKnowledgeBase(
    path="./markdown_files",
    vector_db=vector_db,
    search_type=SearchType.hybrid
)

INFO Creating table: langchain

[2025-09-04T07:35:01Z WARN  lance::dataset::write::insert] No existing dataset at /home/iraitz/TheBridge/B2B/DS4B2B/M5 - Aplicaciones de GenAI/lancedb/langchain.lance, it will be created


Las consultas pueden hacerse tanto por el texto que contienen de cara a buscar las palabras clave, como por el embedding generado.

In [25]:
query_text = "What are agents?"

query = {
    "text" : query_text,
    "vector" : GeminiEmbedder().get_embedding(query_text)
}
query['vector'][:10]

[-0.025601903,
 -0.015883127,
 -0.006648923,
 -0.057227448,
 -0.007338925,
 0.021304669,
 -0.00434841,
 0.010607597,
 -0.0051602297,
 0.030773513]

Como en cualquier proceso ETL, deberemos cargar la información primero.

In [26]:
knowledge_base.load()

INFO Loading knowledge base

INFO Reading: markdown_files/agents.md

INFO Added 3 documents to knowledge base

Y ahora definir un agente enlazado a esa base de conocimiento. Otros sistemas con algo más complejos pero Agno nos ofrece una integración sencilla.

In [28]:
from agno.agent import Agent
from agno.models.google import Gemini

# Create an Agent
agent = Agent(
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    knowledge=knowledge_base,
    tools=[search],
    # search_knowledge=True gives the Agent the ability to search on demand
    # search_knowledge is True by default
    search_knowledge=True,
    instructions=[
        "Include sources in your response.",
        "Always search your knowledge before answering the question.",
        "Only include the output in your response. No other text.",
    ],
    show_tool_calls=True,
    markdown=True,
    add_references=True,
)

agent.print_response("What are agents?", stream=True)

Output()

INFO Found 3 documents

## Multimodelo

Algunas plataformas operacionales se han liado la manta al la cabeza y nos ofrecen sistemas multimodelo que puedan cubrir con todas nuestras necesidades. Este es el caso de [SurrealDB](https://surrealdb.com/) que recientemente ofrece búsqueda basadas en vectores y está integrada con nuestro framework de forma nativa: https://docs.agno.com/vectordb/surrealdb

In [38]:
import os
from surrealdb import Surreal

# SurrealDB connection parameters
SURREALDB_URL = os.getenv("SURREAL_HOST")
SURREALDB_USER = os.getenv("SURREAL_USER")
SURREALDB_PASSWORD = os.getenv("SURREAL_PASS")
SURREALDB_NAMESPACE = os.getenv("SURREAL_NAMESPACE")
SURREALDB_DATABASE = os.getenv("SURREAL_DB")

# Create a client
client = Surreal(url=SURREALDB_URL)
client.signin({"username": SURREALDB_USER, "password": SURREALDB_PASSWORD})
client.use(namespace=SURREALDB_NAMESPACE, database=SURREALDB_DATABASE)

Dado el esquema multimodelo y versátil que proporciona SurrealDB, podemos realizar búsquedas complejas que tienen en cuenta tanto la cercanía semántica como la relación en grafo entre fragmentos. El algoritmo Hierarchical Navigable Small World (HNSW) es una técnica basada en grafos para la búsqueda aproximada de vecinos más cercanos utilizada en muchas bases de datos vectoriales. La búsqueda de vecinos más cercanos sin un índice implica calcular la distancia de la consulta a cada punto de la base de datos, lo que, para conjuntos de datos grandes, resulta computacionalmente prohibitivo.

https://www.pinecone.io/learn/series/faiss/hnsw/

In [39]:
from agno.vectordb.surrealdb import SurrealDb
from agno.embedder.google import GeminiEmbedder
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase

# Database
surrealdb = SurrealDb(
    client=client,
    collection="recetas",  # Collection name
    efc=150,  # HNSW construction time/accuracy trade-off
    m=12,  # HNSW max number of connections per element
    search_ef=40,  # HNSW search time/accuracy trade-off
    embedder=GeminiEmbedder(),
)

# Knowledge base
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=surrealdb
)
knowledge_base.load(recreate=True)

INFO Dropping collection

INFO Creating collection

INFO Loading knowledge base

INFO Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO Added 14 documents to knowledge base

In [40]:
from agno.agent import Agent
from agno.models.google import Gemini

# Create agent and query synchronously
agent = Agent(
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    knowledge=knowledge_base,
    show_tool_calls=True
)
agent.print_response(
    "Cuales son las tres categorías que Thai SELECT establece?",
    markdown=True,
    stream=True
)

Output()